In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
import time
import pandas as pd
from datasets import load_dataset
import os

/Users/nevidujayatilleke/Documents/MSC - Research/OCR Comparative Analysis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# --- Configuration ---
SUBSA_URL = "https://ocr.subasa.lk"

In [4]:
# Path to your webdriver
WEBDRIVER_PATH = "/opt/homebrew/bin/chromedriver"  # Update this

In [5]:
# --- Load your dataset ---
dataset = load_dataset("Ransaka/sinhala_synthetic_ocr-large")
results = []

In [6]:
# --- Initialize WebDriver ---
service = ChromeService(executable_path=WEBDRIVER_PATH)
driver = webdriver.Chrome(service = service)

In [7]:
try:
    driver.get(SUBSA_URL)

    for i, item in enumerate(dataset['train']):
        image = item['image']
        reference_text = item['text']
        print(f"Reference: {reference_text}")
        temp_image_path = f"temp_image_{i}.png"
        image.save(temp_image_path)
        
        try:
            # --- Locate upload element (inspect Subasa's HTML) ---
            upload_input = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='stDropzoneInput']"))
            )
            # time.sleep(0.5)
            # print("Uploading file...")
            upload_input.send_keys(os.path.abspath(temp_image_path))
            # print("Uploaded file")

            # if reference_text == "මේ තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අපිටත් ලගා කරලා දුන්නට දිනේශ් අයියට අනේක වාරයක්":
            #     break

            # --- Locate and click OCR button (inspect Subasa's HTML) ---
            # ocr_button = driver.find_element(By.XPATH, "//button[text()='Process Image']") # Example XPath

            # Add a small delay to allow file to be selected
            time.sleep(0.5)
            # ocr_button.click()

            # --- Wait for result (inspect Subasa's HTML for the result element) ---
            wait = WebDriverWait(driver, 20)
            result_element = wait.until(EC.presence_of_element_located((By.XPATH, """//*[@id="root"]/div[1]/div[1]/div/div/div/section/div[1]/div/div/div/div[13]/div/div/p""")))
            # result_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='stMarkdownContainer p']")))

            # --- Extract text ---
            generated_text = result_element.text.strip()
            print(f"Generated: {generated_text}")
            results.append({'reference': reference_text, 'generated_subasa': generated_text})
            delete_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='fileDeleteBtn'] button")))
            delete_button.click()
            time.sleep(1)

        except Exception as processing_error:
            print(f"Error processing image: {processing_error}")

        finally:
            if os.path.exists(temp_image_path):
                os.remove(temp_image_path)


except Exception as main_error:
    print(f"An error occurred: {main_error}")

finally:
    driver.quit()

Reference: ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම් 1ක් සොයා ගත් අතර ඔහු
Generated: ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම්‌ 1ක් සොයා ගත් අතර ඔහු
Reference: මේ තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අපිටත් ලගා කරලා දුන්නට දිනේශ් අයියට අනේක වාරයක්
Generated: මේ තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අරිටත් ලගා කරලා දුන්නට දිනේශ් අයියට අනේක වාරයක්‌
Reference: සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩියෝව අහන්න පුළුවන්. හැබැයි
Generated: සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩියෝව අහන්න පුළුවන්‌. හැබැයි
Reference: ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය සම්පුර්ණ වු විට මැතිවරණයක්
Generated: ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය සම්පුර්ණ වු විට මැතිවරණයක්‌
Reference: කාර්යාලීය ඇඳුම්, හවස්වරුවේ සාද සඳහා ඇදුම්, ලිනන් හා ඩෙනිම් ඇඳුම්, යට ඇඳුම් පරාසයක් පවතින අතර අවශ්ය
Generated: 211[1181[:1 (128, (1):1(31(1/1ඩ] (1ලි (3[(201]1] අ, [බෝ (111 [10115] 21126], [5] (1/[28] [11:01 1610] (අර1(1 £අ:1(115
Reference: නිවනට යන්න තෘශ්ණාවක් ඕනේ ඒක කුසලමූල තන්හාව.. ඒක අනාස්‍රව එකක් ඇසුරු කල
Generated: නිවනට 

In [8]:
# --- Create DataFrame ---
df_subasa = pd.DataFrame(results)
print(df_subasa.head())
df_subasa.to_csv("subasa_selenium_ocr_results.csv", index=False, encoding='utf-8')

                                           reference  \
0  ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම් 1ක් සොයා ගත්...   
1  මේ තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අපිටත් ල...   
2  සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩි...   
3  ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය ස...   
4  කාර්යාලීය ඇඳුම්, හවස්වරුවේ සාද සඳහා ඇදුම්, ලින...   

                                    generated_subasa  
0  ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම්‌ 1ක් සොයා ගත...  
1  මේ තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අරිටත් ල...  
2  සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩි...  
3  ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය ස...  
4  211[1181[:1 (128, (1):1(31(1/1ඩ] (1ලි (3[(201]...  


In [9]:
df_subasa.head()

,reference,generated_subasa
0,ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම් 1ක් සොයා ගත්...,ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම්‌ 1ක් සොයා ගත...
1,"මේ තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අපිටත් ල...","මේ තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අරිටත් ල..."
2,සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩි...,සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩි...
3,ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය ස...,ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය ස...
4,"කාර්යාලීය ඇඳුම්, හවස්වරුවේ සාද සඳහා ඇදුම්, ලින...","211[1181[:1 (128, (1):1(31(1/1ඩ] (1ලි (3[(201]..."
